In [355]:
#Importing the required libraries.
import pandas as pd #for dataframes and reading csv files.
import numpy as np # for math operations(sqrt)
import matplotlib.pyplot as plt #for plotting the graphs



In [356]:
file_day_od="data/day_od.csv" #directory of the file(file_path)
#csv (comma seperated values, delimiter is comma)
data_day_od=pd.read_csv(file_day_od,sep=",",encoding='utf-16', header=0)
                       
#quoting=2,quotechar='"',escapechar='\r' )
#comment: explicit header=0 to replace column names;  quoting=2 "Non-Numerical"; es

#seperate the columns by name
headerName=["MONTH","DOW","ORIGIN","DESTINATION","CUST_CLASS","COD_COUNTRY","COD_PRO","PRO_COM","FLOW"]
data_day_od.columns=headerName
day_od=pd.DataFrame(data_day_od)#dataFrame of the file(can be read more easily than csv)


# Getting DataFrame for distinct_users_day.csv
file_distinct_users_day="data/distinct_users_day.csv"
data_distinct_users_day=pd.read_csv(file_distinct_users_day,sep=",",encoding="latin-1")
                                    #,quoting=3,escapechar="\"",encoding="latin-1")
distinct_users_day=pd.DataFrame(data_distinct_users_day)
# Getting DataFrame for codici_istat_comune.csv
file_codici_istat_comune="data/codici_istat_comune.csv"
data_codici_istat_comune=pd.read_csv(file_codici_istat_comune,sep=",",delimiter=',',header=0,quoting=3,escapechar="\"",encoding="latin-1")
codici_istat_comune=pd.DataFrame(data_codici_istat_comune)
#codici_istat_comune

# Getting DataFrame for codici_istat_provincia.csv
file_codici_istat_provincia="data/codici_istat_provincia.csv"
data_codici_istat_provincia=pd.read_csv(file_codici_istat_provincia,sep=",",header=0,quoting=3,escapechar="\"",encoding="latin-1")
codici_istat_provincia=pd.DataFrame(data_codici_istat_provincia)
#codici_istat_provincia

# Getting DataFrame for codici_nazioni.csv
import csv
file_codici_nazioni="data/codici_nazioni.csv"
data_codici_nazioni=pd.read_csv(file_codici_nazioni,delimiter=',',header=0,quoting=2, 
                                error_bad_lines=True,encoding="utf-16",quotechar = '"',escapechar='\r')
codici_nazioni=pd.DataFrame(data_codici_nazioni)
 #codici_nazioni[108:112]

## POINT 4

 Study of the visitors' fluxes: let's assume to be asked to provide indications of how to invest resources to improve the mobility toward Padova. Defined the three main directions of visitors and commuter getting to Padova by the Italian highways (sud: A13 toward Bologna-Roma / west: A4 toward Milano-Torino / north-east: A4 toward Venice-Trieste), evaluate which of the three directions has to be prioritized:
 1. Consider a simplified case involving only the mid-range mobility, based on the number of visitors/commuters from the nearby regions only
 * Consider the provinces located on the three directions that are mostly contributing to the flow of weekend visitors and working daily commuters by performing a more detailed study of the fluxes based on the day of the week
 * _[OPEN]_ Use the data available to provide what you believe is the best possible answer 

In [357]:
#import da inserire 

import plotly.graph_objs as go
import plotly.figure_factory as ff

## 4.1 

In this first part, to study the visitor's fluxes from the nearby regions only, we've selected one region for each of the different directions toward Padova:

-Lombardia at west, associated with the A4 toward Milano-Torino <br>
-Emilia Romagna at south, associated with the A13 toward Bologna-Roma <br>
-Friuli Venezia Giulia at east, associated with the A4 toward Venice-Trieste. <br>

for this regions we can be quite sure that the people coming toward Padova are taking the highways of interest, if using the car. 


In [362]:
#DATA FOR THE REGIONS OF INTEREST
#For the regions of interest insert the region capital names and the region names
cap=['Milano', 'Bologna', 'Trieste']
reg=['Lombardia', 'Emilia Romagna', 'Friuli Venezia Giulia']

#Function to obtain the info of the regions of interest 
def info_regions(codici_istat_provincia, capoluogo):
    cod_reg=codici_istat_provincia[codici_istat_provincia['PROVINCIA'] == capoluogo]['COD_REG'].values[0] 
    info_reg=codici_istat_provincia[codici_istat_provincia['COD_REG'] == cod_reg]
    return info_reg

#Create a dictionary of dataframes for the different regions (cod info only)
d = {} 
cod_int=[]
for c in cap:
    d[reg[cap.index(c)]] = pd.DataFrame(info_regions(codici_istat_provincia,c))
    #dataframe d[key] with key=reg[i], data: |COD_REG|COD_PRO|PROVINCIA|PROV_SIGLA
    

#Returns a dataset with all the info (data_day_od) of the regions of interest
def data_regions(data_day_od, cod_all_prov):
    data_reg=data_day_od[data_day_od['COD_PRO'].isin(cod_all_prov)]
    return data_reg

#Define a dictionary with the dataframe for the different regions with all the info from data_day_od
#data_reg[reg[i]]  MONTH|DOW|ORIGIN|DESTINATION|CUST_CLASS|COD_CONTRY|COD_PRO|PRO_COM|FLOW
data_reg = {} 
for i in range(len(reg)):
    data_reg[reg[i]] = pd.DataFrame(data_regions(data_day_od,d[reg[i]]['COD_PRO'].values[:]))
    

##STUDY OF THE FLUXES OF NEARBY REGIONS TO FIND WHICH OF THE THREE DIRECTIONS HAS TO BE PRIORITIZE

#Function to calculate the total flow of the regions of interest 
def flow_regions(info_data, cod_all_prov):
    flow_sum=info_data.groupby(info_data['COD_PRO'].isin(cod_all_prov))['FLOW'].sum()
    flow_sum=flow_sum.reset_index()
    return flow_sum['FLOW'][flow_sum['COD_PRO'] == True][0]
    
#Create a dictionary for the the total flux for the different regions 
d_flow = {}
for i in range(len(reg)):
    d_flow[reg[i]] = flow_regions(data_reg[reg[i]], d[reg[i]]['COD_PRO'].values[:])
    
    
#Dataframe with the total flux for the regions of interest, flow: REGION|FLOW|
flow= pd.DataFrame(d_flow.items(), columns=['REGION', 'FLOW']).sort_values(by='FLOW', ascending=False)


#Display the results
table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]
fig = ff.create_table(flow, height_constant=80, colorscale=table_color) # font_colors=font)#colorscale

color_dict = ['#b32d00', '#fb9804' ,'#b5de2b']
fig.add_trace(go.Bar(x = flow["REGION"], y = flow["FLOW"],marker_color=color_dict,name='Flow',xaxis='x2', yaxis='y2',))
    
fig.update_layout(title_text = 'Total flow for the nearby regions',margin = {'t':60, 'b':80, },
    xaxis = {'domain': [0, .4]},
    xaxis2 = {'domain': [0.5, 1.], 'title':'Region'},
    yaxis2 = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'}
)

fig.show()

    

As we can see from the plot above the region with the highest flow is Lombardia, followed by Emilia Romagna and Friuli Venezia Giulia. Of the three directions it should be therefore proritized the Italian highways A4, torward Milano-Torino.


## 4.2

We’ll now perform a more detailed analysis selecting only the most contributing provinces for each of the selected regions. We'll study first the contribute of each province to the flow and then study the flow trend based on the days of the week. 


In [363]:
# 4.2.1 STUDY OF THE FLUXES OF THE MOSTLY CONTRIBUTING PROVINCES BASED ON THE DAY OF THE WEEK 

#Selection of the provinces that are mostry contributing to the flow

#Function to find the mostly contributing provinces for each region:
#Returns the total flux for each province and the code of the selected provinces
def flow_province(data_regions):
    flow_province=data_regions.groupby('COD_PRO').FLOW.sum()
    flow_sorted=flow_province.sort_values(ascending=False)
    flow_selected=flow_sorted.head(n=3).reset_index()  #Selected the first 3 with the highest flux
    cod_pro_selected=flow_selected['COD_PRO'].values[:]
    return flow_selected, cod_pro_selected


flow_pro={} #Dictionary with a flow dataframe for the different regions
pro_selected={}  #Dictionary with the code of the selected provinces

for i in range(len(reg)): 
    flow_pro[reg[i]],  pro_selected[reg[i]] = flow_province(data_reg[reg[i]]) 
    

#Associating to each province code the province name
#flow_pro: flow for the provinces of a region flow_pro[reg[i]]: 'COD PRO', 'NAME', 'FLOW'
for i in range(len(reg)):
    name_l=[]
    for cod in flow_pro[reg[i]]['COD_PRO'].values[:]:
        if cod in d[reg[i]]['COD_PRO'].values[:]:
            where=np.argwhere(d[reg[i]]['COD_PRO'].values[:]==cod)
            idx=where[0][0] 
            name=d[reg[i]]['PROVINCIA'].values[idx:idx+1][0]
            name_l.append(name)
    flow_pro[reg[i]].insert(1, "NAME", name_l)

    
#Display the results
table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]
color_dict = ['#b32d00', '#fb9804' ,'#b5de2b', 'yellow']


#Plot flow for the provinces
fig_fp = go.Figure()
for i in range(len(reg)):  
        fig_fp.add_trace(go.Bar(x = flow_pro[reg[i]]["NAME"], y = flow_pro[reg[i]]["FLOW"], name=reg[i], marker_color=color_dict[i]))
        fig_fp.update_layout(barmode='relative')
        fig_fp.update_layout(title={'text': "Flow for the mostly contributing provinces",'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},xaxis = {'domain': [0.1, 1.], 'title':'Province'}, yaxis = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'})

fig_fp.show()

#Dataframe
for i in range(len(reg)):  
    fig = ff.create_table(flow_pro[reg[i]],colorscale=table_color)
    fig.update_layout(xaxis = {'domain': [.2, .7]}, yaxis1 = {'domain': [0, 1]})
    fig.update_layout(title={'text': str(reg[i]),'xanchor': 'left','yanchor': 'top'}, xaxis1 = {'domain': [0.3, .7]}, yaxis1 = {'domain': [0., 1]})
    fig.show()
     
    
#Display the new total flow
tot_flow=[]
for i in range(len(reg)):
    tot_flow.append(flow_pro[reg[i]].FLOW.sum())

new_tot_flow=go.Figure()
new_tot_flow.add_trace(go.Bar(x =reg, y=tot_flow, marker_color=color_dict, name='Flow'))
new_tot_flow.update_layout(title={'text': "Total flow of the regions with only the mostrly contributing provinces",'y':0.8,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                   xaxis = {'domain': [0.1, 1.], 'title':'Region'}, 
                                   yaxis = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'},
                                    margin=dict(l=130, r=160, t=150, b=100))

new_tot_flow.show()




From the first plot of the flow for the mostly contributing provinces , we can see that the reason why Lombardia has such a high flow is connected to the province of Milan. Emilia Romagna and Friulia Venezia Giulia on the other hand have a similar flow, with Bologna and Udine, respectively, as the most contributing provinces. Let's also notice that selecting only three provinces for region implies a little higher total flux for Friuli Venezia Giulia instead of Emilia Romagna.

In [364]:
#4.2.2 STUDY OF THE FLUXES BASED ON THE DAY OF THE WEEK

#Returns a dataframe with all the info (from data_day_od) for the selected provinces for each region
def data_selected_province(data_day_od,cod_pro_selected):
    data_province_selected=data_day_od[data_day_od['COD_PRO'].isin(cod_pro_selected)] 
    return data_province_selected

#Define a dictionary with a dataframe for the different regions, containg all the info of the selected provinces 
d_selected_province = {} 
for i in range(len(reg)):
    d_selected_province[reg[i]] = pd.DataFrame(data_selected_province(data_day_od,flow_pro[reg[i]]['COD_PRO'].values[:]))
    
#Sum of the fluxes based on the day of the week and the province
def flow_days(data_province_selected, cod_pro_selected):
    flow_days=data_province_selected.groupby(['DOW', 'COD_PRO']).FLOW.sum().reset_index()
    return flow_days

#Dictrionary with a dataframe for each region, with days information
d_flow_days = {} 
for i in range(len(reg)):
    d_flow_days[reg[i]] = pd.DataFrame(flow_days(d_selected_province[reg[i]],flow_pro[reg[i]]['COD_PRO']))

#Printing the dataframe associating the name to each province code
for i in range(len(reg)):
    name_list=[]
    for cod in d_flow_days[reg[i]]['COD_PRO'].values[:]:
        if cod in d[reg[i]]['COD_PRO'].values[:]:
            where=np.argwhere(d[reg[i]]['COD_PRO'].values[:]==cod)
            idx=where[0][0] #torna poco questo 00
            name=d[reg[i]]['PROVINCIA'].values[idx:idx+1][0]
            name_list.append(name)  
    d_flow_days[reg[i]].insert(1, "NAME", name_list)

print("\n")

#Analysis based on the days
days = ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì', 'Sabato', 'Domenica']

#Function to order the dataset based on the days of the week 
def days_order(dataset,days):
    dataset['DOW']=pd.Categorical(dataset['DOW'], categories=days, ordered=True)
    dataset_days=dataset.sort_values(['DOW'], ignore_index=True)
    return dataset_days

#Function to sort the data by provinces
def sort(dataset):
    dataset=dataset.sort_values(['COD_PRO', 'NAME'],ascending=False, ignore_index=True)
    return dataset

#Dictionary with a dataframe for each region
#Different dataframes based on different type of data organization
flow_each_pro={}

for i in range(len(reg)):
    list_k=[0]
    flow_each_pro[reg[i]]=sort(d_flow_days[reg[i]])
    for k in range(1,len(flow_each_pro[reg[i]]['COD_PRO'])):
        if flow_each_pro[reg[i]]['COD_PRO'][k]!=flow_each_pro[reg[i]]['COD_PRO'][k-1]:
            list_k.append(k)
    list_k.append(len(flow_each_pro[reg[i]]['COD_PRO']))
    for d in range(len(list_k)-1): 
            flow_each_pro[reg[i],d]= pd.DataFrame(days_order(flow_each_pro[reg[i]][list_k[d]:list_k[d+1]],days))
    flow_each_pro[reg[i]]=pd.DataFrame(days_order(d_flow_days[reg[i]],days))
    
    
#Funtion to sum the fluxes of a day for each region 
def flux_sum_day(flow_each_pro):
    flux_sum_pd=flow_each_pro.groupby(['DOW'])['FLOW'].sum().reset_index()
    return flux_sum_pd

#Dictionary with dataframes for each region with the sum of fluxes based on the day of the week
flux_sum_perday={}
for i in range(len(reg)):
        flux_sum_perday[reg[i]]=flux_sum_day(flow_each_pro[reg[i]])
        

#Display the results
color_dict = ['#b32d00', '#fb9804' ,'#b5de2b', 'yellow']
table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]   


#Results for the single regions based on the days of the week
#Plot

concat_tot=pd.concat(flux_sum_perday.values(), axis=0)
color_t=['#e61100','#ff1a1a' ]


fig_regdf = go.Figure()
for i in range(len(reg)):  
        fig_regdf = fig_regdf.add_trace(go.Scatter(x = flux_sum_perday[reg[i]]["DOW"], y = flux_sum_perday[reg[i]]["FLOW"],name =reg[i], marker=dict(color=color_dict[i],size=8,line=dict(color=color_dict[i],width=8))))
        fig_regdf.update_layout(title={'text': "Flow for the selected regions based on the days of the week",'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},xaxis = {'domain': [0.1, 1.], 'title':'Day of the week'}, yaxis = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'})
fig_regdf.add_trace(go.Scatter(x = flux_sum_perday[reg[0]]["DOW"], y = concat_tot.groupby(by='DOW').FLOW.sum(),name ='Total', marker=dict(size=8,line=dict(width=8))))
fig_regdf.show()


##Print dataset if needed
#for i in range(len(reg)):  
#    fig_tab_regdf = ff.create_table(flux_sum_perday[reg[i]],colorscale=table_color)
#    fig_tab_regdf.update_layout(title={'text': str(reg[i]),'xanchor': 'left','yanchor': 'top'}, xaxis1 = {'domain': [0.3, .8]}, yaxis1 = {'domain': [0., 1]})
#    fig_tab_regdf.show()
#    print("\n\n")
    
    
#Results for each province based on the days of the week
#Plot
fig_prodf = go.Figure()
color_dict = [ ['#e61100','#ff1a1a','#ff9f80'], ['#ffff66','#ff8000', '#ffd480'], [ '#b5de2b','#4dff4d','#00cc00' ]]

for i in range(len(reg)):  
    for k in range(len(flow_pro[reg[i]]['COD_PRO'])):
        fig_prodf = fig_prodf.add_trace(go.Bar(x = flow_each_pro[reg[i],k]["DOW"], y = flow_each_pro[reg[i],k]["FLOW"], name = flow_each_pro[reg[i],k].iloc[0,1], marker_color=color_dict[i][2-k]))
        fig_prodf.update_layout(title={'text': "Flow for the mostly contributing provinces based on the days of the week",'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},xaxis = {'domain': [0.1, 1.], 'title':'Day of the week'}, yaxis = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'})
fig_prodf.show()


##Print dataset if needed    
#for i in range(len(reg)):  
#    fig_tab_prodf = ff.create_table(flow_each_pro[reg[i]],colorscale=table_color, name=reg[i])
#    fig_tab_prodf.update_layout(title={'text': str(reg[i]),'xanchor': 'left','yanchor': 'top'}, xaxis1 = {'domain': [0.3, .8]}, yaxis1 = {'domain': [0., 1]})
#    fig_tab_prodf.show()
#    print("\n\n")

In the first plot we can see the trend for the three regions. For Lombardia we have an increasing flow going into the weekends with a peak on Saturday. Friuli Venezia Giulia has instead an opposite trend with a higher flux during the week days and a drop  on Saturday and Sunday. For Emilia Romagna we have quite a constant flow with a peaks on Wednesday.  Looking at total flow we can see a lower value on the weekends, suggesting a mobility linked to  commuters during the weekdays.  <br>

In the second plot we can see that Milan has the highest flow, at least two times bigger than the others provinces. The trend for Milano explains the Lombardia’s trend while Brescia and Mantova have an opposite trend with a lower flow on the weekends As regards Emilia Romagna, the trend for Bologna is in agreement with the one of the entire region, while for Ferrara the flux gets smaller on the weekends.For Friuli Venezia Gulia the contribution to the flow is mostly given by Pordenone and Udine qith a higher flow during the week. 

## 4.3

In this part we want to see what happens if we consider more provinces for each one of the three directions. We are also interested to see what happens if we consider provinces that belongs to the region of Veneto.

In [352]:
#For the regions of interest insert the region capital names and the region names
m_cap=['Milano', 'Bologna', 'Trieste', 'Roma', 'Torino', 'Firenze', 'Napoli','Trento','Venezia']
m_reg=['Lombardia', 'Emilia Romagna', 'Friuli Venezia Giulia', 'Lazio', 'Piemonte', 'Toscana', 'Campania','Trentino Alto Adige','Veneto']

#Dataframes for the different regions (cod info only)
m_d = {} 
for c in m_cap:
    m_d[m_reg[m_cap.index(c)]] = pd.DataFrame(info_regions(codici_istat_provincia,c))

#Dataframes for the different regions with all the info from data_day_od
m_data_reg = {} 
for i in range(len(m_reg)):
    m_data_reg[m_reg[i]] = pd.DataFrame(data_regions(data_day_od,m_d[m_reg[i]]['COD_PRO'].values[:]))
    
code_padua=codici_istat_provincia[codici_istat_provincia['PROVINCIA']=='Padova']['COD_PRO'].values[0]


#STUDY based on the provinces and days of the week (as 4.2)

#Function to find the mostly contributing provinces for each region
#Returns the total flux for each province and the code of the selected provinces
#Padua will not be considered 
def flow_province(data_regions):
    flow_province=data_regions.groupby('COD_PRO').FLOW.sum()
    flow_sorted=flow_province.sort_values(ascending=False)
    flow_selected=flow_sorted.loc[flow_sorted[:] >= 1e4].reset_index() #Selection based on the flux
    for cod in flow_sorted.reset_index()['COD_PRO'].values[:]:
        if cod == code_padua:
            flow_selected=flow_sorted.loc[flow_sorted[:] >= 1e4].reset_index()
            flow_selected=flow_selected.loc[1:] #not considering Padua
    cod_pro_selected=flow_selected['COD_PRO'].values[:]
    return flow_selected, cod_pro_selected


m_flow_pro={} 
m_pro_selected={} 

for i in range(len(m_reg)): 
    m_flow_pro[m_reg[i]], m_pro_selected[m_reg[i]] = flow_province(m_data_reg[m_reg[i]]) 


#Associating each province code the province name
#flow of the province for a region flow_pro[reg[i]]: 'COD PRO', 'NAME', 'FLOW'
for i in range(len(m_reg)):
    name_l=[]
    for cod in m_flow_pro[m_reg[i]]['COD_PRO'].values[:]:
        if cod in m_d[m_reg[i]]['COD_PRO'].values[:]:
            where=np.argwhere(m_d[m_reg[i]]['COD_PRO'].values[:]==cod)
            idx=where[0][0] 
            name=m_d[m_reg[i]]['PROVINCIA'].values[idx:idx+1][0]
            name_l.append(name)
    m_flow_pro[m_reg[i]].insert(1, "NAME", name_l)
     
    
#Display the results
table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]
color_dict=['#b32d00', '#fb9804' ,'#b5de2b','#6ece8c', '#9ce2bf','#94c0aa','#39604d','#2277a4', 'blue']

#Plot no Veneto
fig_morep = go.Figure()
for i in range(len(m_reg)-1):  
        fig_morep.add_trace(go.Bar(x = m_flow_pro[m_reg[i]]["NAME"], y = m_flow_pro[m_reg[i]]["FLOW"], name=m_reg[i], marker_color=color_dict[i]))
        fig_morep.update_layout(barmode='relative')
        fig_morep.update_layout(title={'text': "Flow for the selected provinces",'y':0.9,'x':0.45,'xanchor': 'center','yanchor': 'top'},
                                xaxis = {'domain': [0.1, 1.], 'title':'Province'}, 
                                yaxis = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'},
                                margin=dict(l=80, r=180, t=90, b=90))
fig_morep.show()


#Highways no Veneto
more_prov=pd.concat(m_flow_pro.values(), axis=0)
more_prov=more_prov.reset_index()[:-6]
highway=['A4-west','A4-west','A4-west','A4-west','A13','A13','A4-east','A4-east','A13','A4-west','A13','A13','A4-west','A4-west']
more_prov.insert(3,'DIR',highway)  
flow_dir=more_prov.groupby(by='DIR').FLOW.sum().reset_index().sort_values(by='FLOW',ascending=False)

fig_flow_dir = ff.create_table(flow_dir, height_constant=80, colorscale=table_color) # font_colors=font)#colorscale

color_dict_a = ['#b32d00', '#fb9804' ,'#b5de2b']
fig_flow_dir.add_trace(go.Bar(x = flow_dir["DIR"], y = flow_dir["FLOW"], marker_color=color_dict_a,name='Flow',xaxis='x2', yaxis='y2',))
    
fig_flow_dir.update_layout(title_text = 'Total flow for the three highways',margin = {'t':60, 'b':80, },
    xaxis = {'domain': [0, .4]},
    xaxis2 = {'domain': [0.5, 1.], 'title':'Highway'},
    yaxis2 = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'}
)

fig_flow_dir.show()


#Plot with Veneto
fig_allp=go.Figure()
for i in range(len(m_reg)):  
        fig_allp.add_trace(go.Bar(x = m_flow_pro[m_reg[i]]["NAME"], y = m_flow_pro[m_reg[i]]["FLOW"], name=m_reg[i], marker_color=color_dict[i]))
        fig_allp.update_layout(title={'text': "Flow including provinces of Veneto",'y':0.9,'x':0.45,'xanchor': 'center','yanchor': 'top'},
                                xaxis = {'domain': [0.1, 1.], 'title':'Province'}, 
                                yaxis = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'},
                                margin=dict(l=80, r=180, t=90, b=90))
fig_allp.show()


#Highways with Veneto
all_prov=pd.concat(m_flow_pro.values(), axis=0)
highway=['A4-west','A4-west','A4-west','A4-west','A13','A13','A4-east','A4-east','A13','A4-west','A13','A13','A4-west','A4-west',
         'A4-east','A4-west','A4-east','A4-west','A13','A4-east']
all_prov.insert(3,'DIR',highway)  
all_flow_dir=all_prov.groupby(by='DIR').FLOW.sum().reset_index().sort_values(by='FLOW',ascending=False)

table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]
fig_all_flow_dir = ff.create_table(all_flow_dir, height_constant=80, colorscale=table_color) # font_colors=font)#colorscale

color_dict_a = ['#b5de2b','#b32d00', '#fb9804']
fig_all_flow_dir.add_trace(go.Bar(x = all_flow_dir["DIR"], y = all_flow_dir["FLOW"], marker_color=color_dict_a,name='Flow',xaxis='x2', yaxis='y2',))
    
fig_all_flow_dir.update_layout(title_text = 'Total flow for the three highways with Veneto',margin = {'t':60, 'b':80, },
    xaxis = {'domain': [0, .4]},
    xaxis2 = {'domain': [0.5, 1.], 'title':'Highway'},
    yaxis2 = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'}
)

fig_all_flow_dir.show()

    
#Dataframe with all the info
print("\n")
tab_flow_dir_prov= ff.create_table(all_prov,colorscale=table_color)
tab_flow_dir_prov.update_layout(xaxis = {'domain': [.2, .7]}, yaxis1 = {'domain': [0, 1]})
tab_flow_dir_prov.show()  

In the first case we take into consideration also Torino, Trento and Bolzano on the A4-west highway and Firenze, Roma and Napoli for the A13. From the first plot we can see that Roma and Trento are the two new provinces contributing the most, after Milan. Considering then the total flow for the three highways ,it results, once again, that the A4-west should be prioritized being the one with the highest flow. <br>


However, in the second case, we observe that the biggest contribution to the flow is given by the provinces of Veneto, in particular Venezia. From this it follows that the highway with the highest flow is no longer the A4-west but the A4-east, so the one to be prioritized should be the highway section connecting Padova to Venezia. 


## Milocco's Part to add to the 6th part

#We made a cumulative bar plot for each direction and for the different months present in the data.
#As expect, the rank are a4-West, a4-East and a13-South mostly used in April and May.


In [212]:
def flow_month_reg(data_reg):
    flow_m_reg=data_reg.groupby(['MONTH']).FLOW.sum().reset_index()
    return flow_m_reg

d_flow_month_reg = {} 

for i in range(len(reg)):
    d_flow_month_reg[reg[i]] = pd.DataFrame(flow_month_reg(data_reg[reg[i]]))
    
def flow_month_pro(data_province_selected, cod_pro_selected):
    flow_m=data_province_selected.groupby(['MONTH', 'COD_PRO']).FLOW.sum().reset_index()
    return flow_m

d_flow_month_pro = {} 

for i in range(len(reg)):
    d_flow_month_pro[reg[i]] = pd.DataFrame(flow_month_pro(d_selected_province[reg[i]],flow_pro[reg[i]]['COD_PRO']))


#reconcatenate
df=d_flow_month_pro[reg[0]].merge(flow_pro[reg[0]], on="COD_PRO")
df["Regions"]=reg[0]
for i in range(1,len(reg)):
    df=pd.concat([df, d_flow_month_pro[reg[i]].merge(flow_pro[reg[i]], on="COD_PRO")], ignore_index=True)
    df=df.replace(np.nan, reg[i])
df.rename(columns={"FLOW_x":"FLOW", "NAME":"Province", "FLOW_y":"FLOW TOTAL"}, inplace=True)


df["MONTH"] = pd.Categorical(df["MONTH"],
                    categories=["Febbraio","Marzo","Aprile","Maggio"])
df.sort_values(by=["MONTH","FLOW"], inplace=True, ignore_index=True)
total=d_flow_month_pro["Lombardia"].FLOW.sum()


#Plot
color=['#b32d00', '#fb9804' ,'#b5de2b'][::-1]

import plotly.express as px
fig = px.bar(df, x = df["MONTH"], y = df["FLOW"], color=df["Regions"], color_discrete_sequence=color, text=df["Province"], title="Visitors of nearby regions on months")
#fig.update_xaxes(categoryorder="total ascending")
fig.show()

fig = px.bar(df, x = df["Regions"], y = df["FLOW"], color=df["MONTH"], color_discrete_sequence=px.colors.qualitative.D3, text=df["Province"], title="Visitors of nearby regions on months")
fig.update_xaxes(categoryorder="total ascending")
fig.show()


#Display the data
#tab_months= ff.create_table(df,colorscale=table_color)
#tab_months.update_layout(xaxis = {'domain': [.1, .9]}, yaxis1 = {'domain': [0, 1]})
#tab_months.show()  
